In [19]:
#Librerias a usar
import pandas as pd
import gzip

Primera función

se habren los datasets que ya se les realizo ETL y se guardaron como csv

In [20]:
steam_games = pd.read_csv('../datasets/steam_games.csv')


In [21]:
with gzip.open('../datasets/user_items.csv.gz', 'rt', encoding='utf-8') as f:
    user_items = pd.read_csv(f)

Se seleccionan solo las columnas que se van a usar para realizar la funcion

In [22]:
steam_games.head(0)

,publisher,genres,app_name,title,url,release_date,reviews_url,specs,price,early_access,id,developer,year


In [23]:
steam_games_f1 = steam_games[['id', 'genres', 'year']]

In [24]:
steam_games_f1.head(2)

,id,genres,year
0,761140,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",2018
1,643980,"['Free to Play', 'Indie', 'RPG', 'Strategy']",2018


In [25]:
user_items.head(0)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks


In [26]:
user_items_f1 = user_items[['item_id', 'playtime_forever']]

In [27]:
user_items.shape

(5153209, 8)

In [28]:
user_items_f1.head()

,item_id,playtime_forever
0,10,6.0
1,20,0.0
2,30,7.0
3,40,0.0
4,50,0.0


In [29]:
# 1. Fusionar los DataFrames
merged_df = pd.merge(steam_games_f1, user_items_f1, left_on='id', right_on='item_id')

In [30]:
merged_df.head()

,id,genres,year,item_id,playtime_forever
0,282010,"['Action', 'Indie', 'Racing']",1997,282010,5.0
1,282010,"['Action', 'Indie', 'Racing']",1997,282010,0.0
2,282010,"['Action', 'Indie', 'Racing']",1997,282010,0.0
3,282010,"['Action', 'Indie', 'Racing']",1997,282010,0.0
4,282010,"['Action', 'Indie', 'Racing']",1997,282010,13.0


ya fusionados los dos df se guadan con csv para que sean consumidos por la función 

In [31]:
merged_df.to_csv('../datasets/data_funcion1.csv',index=False)

In [32]:
with gzip.open('../datasets/data_funcion1.csv.gz', 'wb') as f:
    merged_df.to_csv(f, index=False, encoding='utf-8')

In [33]:
def PlayTimeGenre(genero: str):
    '''Esta función debe devolver año con mas horas jugadas para dicho género.
    Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}'''
    
    # Leer el DataFrame desde el archivo CSV
    merged_df = pd.read_csv('../datasets/data_funcion1.csv.gz')

    # Capitalizar la primera letra del género
    genero = genero.capitalize()

    # Filtrar el DataFrame para obtener solo las filas relacionadas con el género específico
    filtro_df = merged_df[merged_df['genres'].apply(lambda x: genero in x)]

    if not filtro_df.empty:
        # Agrupar por año sumando las horas jugadas para cada año
        grouped_df = filtro_df.groupby('year')['playtime_forever'].sum().reset_index()

        # Encontrar el año con la mayor cantidad de horas jugadas
        max_year = grouped_df.loc[grouped_df['playtime_forever'].idxmax()]['year']

        return {"Año de lanzamiento con más horas jugadas para Género {}:".format(genero): int(max_year)}
    else:
        return {"Género {} no encontrado en el DataFrame.".format(genero): None}


In [34]:

# Ejemplo de uso
resultado = PlayTimeGenre('ACTION')
print(resultado)

{'Año de lanzamiento con más horas jugadas para Género Action:': 2012}


funcion 2

son casi los mismos pasos de la funcion 1, se le agrea la columna user_id 

In [35]:
steam_games_f2 = steam_games[['id', 'genres', 'year',]]

In [36]:
user_items_f2 = user_items[['item_id', 'playtime_forever', 'user_id']]

In [37]:
merged_df2 = pd.merge(steam_games_f2, user_items_f2, left_on='id', right_on='item_id')

In [38]:
merged_df2.head()

,id,genres,year,item_id,playtime_forever,user_id
0,282010,"['Action', 'Indie', 'Racing']",1997,282010,5.0,UTNerd24
1,282010,"['Action', 'Indie', 'Racing']",1997,282010,0.0,I_DID_911_JUST_SAYING
2,282010,"['Action', 'Indie', 'Racing']",1997,282010,0.0,76561197962104795
3,282010,"['Action', 'Indie', 'Racing']",1997,282010,0.0,r3ap3r78
4,282010,"['Action', 'Indie', 'Racing']",1997,282010,13.0,saint556


In [39]:
with gzip.open('../datasets/data_funcion2.csv.gz', 'wb') as f:
    merged_df2.to_csv(f, index=False, encoding='utf-8')

In [40]:
def UserForGenre(genero: str):

    '''Esta función devuelve el usuario que acumula más horas jugadas para el género dado y 
        una lista de la acumulación de horas jugadas por año.
        Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, 
        "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}'''

    # Leer el DataFrame desde el archivo CSV
    merged_df2 = pd.read_csv('../datasets/data_funcion2.csv.gz')

    genero = genero.capitalize()
    # Paso 1: Filtrar el DataFrame para el género dado
    filtered_df = merged_df2[merged_df2['genres'].apply(lambda x: genero in x)]

    if not filtered_df.empty:
        # Paso 2: Encontrar el usuario con más horas jugadas
        user_with_max_hours = filtered_df.groupby('user_id')['playtime_forever'].sum().idxmax()

        # Paso 3: Crear una lista de acumulación de horas jugadas por año
        grouped_df = filtered_df.groupby(['year', 'user_id'])['playtime_forever'].sum().reset_index()
        hours_accumulation = [{'Año': int(year), 'Horas': int(hours)} for year, hours in grouped_df.groupby('year')['playtime_forever'].max().items()]

        # Paso 4: Retornar el resultado en el formato deseado
        result = {"Usuario con más horas jugadas para Género {}".format(genero): user_with_max_hours,
                  "Horas jugadas": hours_accumulation}
        return result
    else:
        return {"Género {} no encontrado en el DataFrame.".format(genero): None}

In [41]:
# Ejemplo de uso
genero_ejemplo = 'action'  # Puedes cambiar esto al género que desees
resultado = UserForGenre(genero_ejemplo)
print(resultado)

{'Usuario con más horas jugadas para Género Action': 'Sp3ctre', 'Horas jugadas': [{'Año': 1983, 'Horas': 331}, {'Año': 1984, 'Horas': 118}, {'Año': 1988, 'Horas': 7678}, {'Año': 1989, 'Horas': 494}, {'Año': 1990, 'Horas': 1424}, {'Año': 1991, 'Horas': 761}, {'Año': 1992, 'Horas': 605}, {'Año': 1993, 'Horas': 22001}, {'Año': 1994, 'Horas': 5539}, {'Año': 1995, 'Horas': 10430}, {'Año': 1996, 'Horas': 7859}, {'Año': 1997, 'Horas': 9214}, {'Año': 1998, 'Horas': 76899}, {'Año': 1999, 'Horas': 150212}, {'Año': 2000, 'Horas': 356381}, {'Año': 2001, 'Horas': 150918}, {'Año': 2002, 'Horas': 27561}, {'Año': 2003, 'Horas': 532817}, {'Año': 2004, 'Horas': 529351}, {'Año': 2005, 'Horas': 81113}, {'Año': 2006, 'Horas': 723682}, {'Año': 2007, 'Horas': 112794}, {'Año': 2008, 'Horas': 231030}, {'Año': 2009, 'Horas': 396428}, {'Año': 2010, 'Horas': 393408}, {'Año': 2011, 'Horas': 416405}, {'Año': 2012, 'Horas': 689709}, {'Año': 2013, 'Horas': 409562}, {'Año': 2014, 'Horas': 358880}, {'Año': 2015, 'Horas

Función 3

importamos los csv a usar 

In [42]:
user_reviews = pd.read_csv('../datasets/user_review.csv')

In [43]:
steam_games = pd.read_csv('../datasets/steam_games.csv')

Seleccionamos las columnas que vamos a usar

In [44]:
steam_games = steam_games[['id','title']]

combinamos los df

In [45]:
merged_df3 = pd.merge(steam_games, user_reviews, left_on='id', right_on='item_id')

In [46]:
merged_df3 = merged_df3[['title', 'year', 'recommend', 'sentiment_analysis']]

Guardamos el df solo con la informacion necesaria para la consulta

In [47]:
with gzip.open('../datasets/data_funcion3_4.csv.gz', 'wb') as f:
    merged_df3.to_csv(f, index=False, encoding='utf-8')
    

In [48]:
def UsersRecommend(anio: int):

    '''Esta función el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = 
    True y comentarios positivos/neutrales) Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]'''

    user_reviews = pd.read_csv('../datasets/data_funcion3_4.csv.gz')
   
    # Paso 1: Filtrar el DataFrame para el año dado
    reviews_anio = user_reviews[user_reviews['year'] == anio]
    
    if reviews_anio.empty:
        return {"mensaje": "No hay revisiones para el año proporcionado."}

    # Paso 2: Filtrar para incluir solo revisiones recomendadas y con sentimiento positivo o neutral
    relevant_reviews = reviews_anio[(reviews_anio['recommend'] == True) & (reviews_anio['sentiment_analysis'].isin([1, 2]))]

    if relevant_reviews.empty:
        return {"mensaje": "No hay revisiones recomendadas y con sentimiento positivo o neutral para el año proporcionado."}

    # Paso 3: Contar la cantidad de recomendaciones para cada juego
    game_recommendation_counts = relevant_reviews.groupby('title')['recommend'].count().reset_index()

    # Paso 4: Ordenar los juegos en función de la cantidad de recomendaciones
    top_games = game_recommendation_counts.sort_values(by='recommend', ascending=False)

    # Paso 5: Seleccionar los tres juegos principales
    top_3_games = top_games.head(3)

    # Paso 6: Crear el formato de retorno deseado
    result = [{"Puesto {}: {}".format(i + 1, title)} for i, title in enumerate(top_3_games['title'][:3])]

    return result

In [49]:
# Ejemplo de uso
resultado = UsersRecommend(2012)
print(resultado)

[{'Puesto 1: Team Fortress 2'}, {'Puesto 2: Terraria'}, {'Puesto 3: Left 4 Dead 2'}]


Funcion 4

Esta fncion es muy similar a la anterior, asi que la copiaremos a adaptaremos a lo que queremos 

In [50]:
def UsersNotRecommend(anio: int):

    '''Esta función el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)    
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]'''

    user_reviews = pd.read_csv('../datasets/data_funcion3_4.csv.gz')
   
    # Paso 1: Filtrar el DataFrame para el año dado
    reviews_anio = user_reviews[user_reviews['year'] == anio]
    
    if reviews_anio.empty:
        return {"mensaje": "No hay revisiones para el año proporcionado."}

    # Paso 2: Filtrar para incluir solo revisiones no recomendadas y con sentimiento negativos
    relevant_reviews = reviews_anio[(reviews_anio['recommend'] == False) & (reviews_anio['sentiment_analysis'] == 0)]

    if relevant_reviews.empty:
        return {"mensaje": "No hay revisiones no recomendadas y con sentimiento negativos para el año proporcionado."}

    # Paso 3: Contar la cantidad de recomendaciones negativas para cada juego
    game_recommendation_counts = relevant_reviews.groupby('title')['recommend'].count().reset_index()

    # Paso 4: Ordenar los juegos en función de la cantidad de recomendaciones
    top_games = game_recommendation_counts.sort_values(by='recommend', ascending=False)

    # Paso 5: Seleccionar los tres juegos principales
    top_3_games = top_games.head(3)

    # Paso 6: Crear el formato de retorno deseado
    result = [{"Puesto {}: {}".format(i + 1, title)} for i, title in enumerate(top_3_games['title'][:3])]

    return result

In [51]:
resultado = UsersNotRecommend(2013)
print(resultado)

[{'Puesto 1: Team Fortress 2'}, {'Puesto 2: Tactical Intervention'}, {'Puesto 3: Counter-Strike: Global Offensive'}]


Funcion 5

In [124]:
steam_games = pd.read_csv('../datasets/steam_games.csv')

In [125]:
user_reviews = pd.read_csv('../datasets/user_review.csv')

In [126]:
steam_games.head(0)

,publisher,genres,app_name,title,url,release_date,reviews_url,specs,price,early_access,id,developer,year


In [127]:
user_reviews.head(0)

,user_id,posted,item_id,recommend,sentiment_analysis,year


In [128]:
user_reviews = user_reviews[['item_id', 'user_id', 'sentiment_analysis']]

In [129]:
steam_games = steam_games[['id', 'year']]

In [130]:
merge_df5 = pd.merge(user_reviews, steam_games, left_on='item_id', right_on='id')

Guardamos en csv

In [145]:
merge_df5.head()

,item_id,user_id,sentiment_analysis,id,year
0,1250,76561197970982479,2,1250,2009
1,1250,76561197970982479,2,1250,2009
2,1250,76561197970982479,2,1250,2009
3,1250,VVillis,2,1250,2009
4,1250,jarrodtrim,0,1250,2009


In [79]:
# Convertir la columna sentiment_analysis a tipo de datos cadena (str)
merge_df5['sentiment_analysis'] = merge_df5['sentiment_analysis'].astype(str)

In [131]:
with gzip.open('../datasets/data_funcion5.csv.gz', 'wb') as f:
    merge_df5.to_csv(f, index=False, encoding='utf-8')

In [148]:
def sentiment_analysis(anio: int):
    '''Esta funcion según el año de lanzamiento, se devuelve una lista con la cantidad de 
    registros de reseñas de usuarios que se encuentren categorizados con un análisis de 
    sentimiento.Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}'''

    merge_df5 = pd.read_csv('../datasets/data_funcion5.csv.gz')

    # Filtrar el DataFrame por el año proporcionado
    relevant_reviews = merge_df5[merge_df5['year'] == anio]

    # Contar la cantidad de registros para cada categoría de análisis de sentimiento
    sentiment_counts = relevant_reviews.groupby('sentiment_analysis').size()

     # Convertir los índices a tipos de datos estándar (int)
    sentiment_counts.index = sentiment_counts.index.astype(int)

    # Crear el diccionario de resultados
    result = {
        'Negative': int(sentiment_counts.get(0, 2)),
        'Neutral': int(sentiment_counts.get(1, 2)),
        'Positive': int(sentiment_counts.get(2, 3))
    }
    

    return  result

In [149]:
resultado = sentiment_analysis(2013)
print(resultado)

{'Negative': 958, 'Neutral': 1239, 'Positive': 4084}
